In [60]:
import os, datetime, pandas as pd
import gmaps
from config import settings
from sqlalchemy import create_engine
from  sqlalchemy.connectors import pyodbc
from sqlalchemy.dialects import postgresql
DT = datetime.datetime.today().strftime('%Y%m%d') #'20200322' #
DATEY = datetime.datetime.today().strftime('%Y-%m-%d') #'2020-03-22' #
pgHost = settings['pgHost']
pgUser = settings['pgUser']
pgPass = os.environ['PGPASS']
pgDb = settings['pgDb']
gmaps.configure(api_key=os.environ["GMAPS_API_KEY"])

pg = create_engine(f"postgresql://{pgUser}:{pgPass}@{pgHost}/{pgDb}")

bubbleSQL = f"""SELECT lng, lat, max(confirmed) confirmed
from daily
where lng>0 or lng<0
--where combinedkey similar to '%%US' and lat>0  and lastupdate<='{DATEY}'
group by lat, lng"""

deathSQL = f"""SELECT lng, lat, max(deaths) deaths
from daily
where lng>0 or lng<0
--where combinedkey similar to '%%US' and lat>0 and lastupdate<='{DATEY}'
group by lat, lng"""

In [61]:
data = pg.execute(bubbleSQL)
data = [d for d in data]
idf = pd.DataFrame(data, columns=['lng','lat','infected'])

In [62]:
data = pg.execute(deathSQL)
data = [d for d in data]
ddf = pd.DataFrame(data, columns=['lng','lat','deaths'])

In [63]:
idf.sort_values('infected')

,lng,lat,infected
1948,-115.001524,47.152840,0
2965,-101.055370,39.351076,0
2963,-100.555663,41.913590,0
679,-88.176575,38.087062,0
1309,-95.743233,37.559372,0
...,...,...,...
1776,-73.971526,40.767273,76876
2011,10.451526,51.165691,107663
1689,2.213700,46.227600,109069
3115,12.567380,41.871940,135586


In [64]:
ddf.sort_values('deaths')

,lng,lat,deaths
0,-85.721427,35.384337,0
2379,-89.802451,33.770362,0
2380,-109.224521,47.263316,0
2381,-95.604068,36.370044,0
2382,-85.800701,43.554374,0
...,...,...,...
1776,-73.971526,40.767273,4009
2981,-3.436000,55.378100,6159
1689,2.213700,46.227600,10328
1872,-3.749220,40.463667,14045


In [111]:
fig = gmaps.figure(map_type='HYBRID', )
death_layer = gmaps.heatmap_layer(ddf[['lat', 'lng']], weights=ddf.deaths, max_intensity=17127, point_radius=15.0)
infected_layer = gmaps.heatmap_layer(idf[['lat', 'lng']], weights=idf.infected, max_intensity=141942, point_radius=25.0)
death_layer.gradient = [(0,0,0, 0.0),'black','red']
fig.add_layer(infected_layer)
fig.add_layer(death_layer)

In [112]:
fig

Figure(layout=FigureLayout(height='420px'))

In [113]:
zipdf = pd.read_csv('C:/Users/ian.rubenstein/repos/coronavirus-data/tests-by-zcta.csv')

In [114]:
zipdf

,MODZCTA,Positive,Total,zcta_cum.perc_pos
0,NaN,4735,4842,97.79
1,10001.0,170,364,46.70
2,10002.0,405,792,51.14
3,10003.0,230,543,42.36
4,10004.0,20,45,44.44
...,...,...,...,...
173,11691.0,822,1226,67.05
174,11692.0,214,332,64.46
175,11693.0,147,242,60.74
176,11694.0,267,457,58.42


In [115]:
import requests

In [119]:
mzip = requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address=11226&key={os.environ['GMAPS_API_KEY']}").json()

In [124]:
mzip['results'][0]['geometry']['location']

{'lat': 40.6470923, 'lng': -73.9536163}

In [128]:
zipdf

,MODZCTA,Positive,Total,zcta_cum.perc_pos,lat,lng
0,NaN,4735,4842,97.79,0.0,0.0
1,10001.0,170,364,46.70,0.0,0.0
2,10002.0,405,792,51.14,0.0,0.0
3,10003.0,230,543,42.36,0.0,0.0
4,10004.0,20,45,44.44,0.0,0.0
...,...,...,...,...,...,...
173,11691.0,822,1226,67.05,0.0,0.0
174,11692.0,214,332,64.46,0.0,0.0
175,11693.0,147,242,60.74,0.0,0.0
176,11694.0,267,457,58.42,0.0,0.0


In [135]:
for i,row in zipdf.iterrows():
    try:
        resp =  requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address={int(row[0])}&key={os.environ['GMAPS_API_KEY']}").json()
        latlng = resp['results'][0]['geometry']['location']
        zipdf.iloc[i,4] = latlng['lat']
        zipdf.iloc[i,5] = latlng['lng']
        print(f"{int(row[0])} - {latlng}")
    except Exception as e:
        print(e)

cannot convert float NaN to integer
10001 - {'lat': 40.75368539999999, 'lng': -73.9991637}
10002 - {'lat': 40.7135097, 'lng': -73.9859414}
10003 - {'lat': 40.7322535, 'lng': -73.9874105}
10004 - {'lat': 40.7038704, 'lng': -74.0138541}
10005 - {'lat': 40.7064363, 'lng': -74.0094562}
10006 - {'lat': 40.709329, 'lng': -74.0131196}
10007 - {'lat': 40.7136487, 'lng': -74.0087126}
10009 - {'lat': 40.7275043, 'lng': -73.9800645}
10010 - {'lat': 40.7388319, 'lng': -73.98153370000001}
10011 - {'lat': 40.7464969, 'lng': -74.0094471}
10012 - {'lat': 40.7250632, 'lng': -73.9976946}
10013 - {'lat': 40.7217861, 'lng': -74.0094471}
10014 - {'lat': 40.7366138, 'lng': -74.0094471}
10016 - {'lat': 40.74727, 'lng': -73.9800645}
10017 - {'lat': 40.7519846, 'lng': -73.9697795}
10018 - {'lat': 40.755322, 'lng': -73.9932872}
10019 - {'lat': 40.7686973, 'lng': -73.9918181}
10021 - {'lat': 40.7700703, 'lng': -73.9580246}
10022 - {'lat': 40.7593941, 'lng': -73.9697795}
10023 - {'lat': 40.7769059, 'lng': -73.980

11427 - {'lat': 40.7302787, 'lng': -73.7442568}
11428 - {'lat': 40.7226473, 'lng': -73.7389596}
11434 - {'lat': 40.6812166, 'lng': -73.7772032}
11432 - {'lat': 40.7154518, 'lng': -73.7919103}
11433 - {'lat': 40.69978340000001, 'lng': -73.7860276}
11435 - {'lat': 40.7031635, 'lng': -73.8095574}
11436 - {'lat': 40.6718171, 'lng': -73.7977928}
11691 - {'lat': 40.6024346, 'lng': -73.762495}
11692 - {'lat': 40.592658, 'lng': -73.7977928}
11693 - {'lat': 40.6166766, 'lng': -73.8272029}
11694 - {'lat': 40.5722132, 'lng': -73.8624893}
11697 - {'lat': 40.5591088, 'lng': -73.9095279}


In [138]:
zipdf=zipdf.dropna()

In [140]:
zipdf.to_csv('C:/Users/ian.rubenstein/repos/coronavirus-data/tests-by-zcta_latlng.csv')

In [147]:
zipdf.sort_values('Positive')

,MODZCTA,Positive,Total,zcta_cum.perc_pos,lat,lng
6,10006.0,8,33,24.24,40.709329,-74.013120
4,10004.0,20,45,44.44,40.703870,-74.013854
43,10280.0,23,66,34.85,40.708427,-74.017348
90,11109.0,29,80,36.25,40.744908,-73.956922
7,10007.0,30,86,34.88,40.713649,-74.008713
...,...,...,...,...,...,...
118,11230.0,1054,1670,63.11,40.619063,-73.965372
73,10467.0,1142,1942,58.81,40.872399,-73.871310
145,11373.0,1142,1663,68.67,40.737975,-73.880130
108,11219.0,1314,1908,68.87,40.631075,-73.997695


In [154]:
fig = gmaps.figure(map_type='HYBRID', )
infected_layer = gmaps.heatmap_layer(zipdf[['lat', 'lng']], weights=zipdf['Positive'], max_intensity=1700, point_radius=20.0)
fig.add_layer(infected_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [1]:
import datetime

In [15]:
datetime.datetime.today().year

2020

datetime.datetime(2020, 5, 1, 0, 0)